In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import json
from PIL import Image
from collections import Counter
import pickle
import shutil

In [2]:
base_path = '..'
generate_all_datset_annots = True

In [ ]:
base_path = '..'
num_folds = 5
generate_masks = True
should_dilate = False
use_merged_dataset = False
generate_all_datset_annots = True
base_data_dir = 'dataset1_files' if not generate_all_datset_annots else 'all_dataset_files'
base_data_name = 'all_dataset1' if not generate_all_datset_annots else 'all_dataset'
get_whole_dataset_into_fold = False
pick_validation_from_file = False
input_imgs_dir = f'{base_path}/{base_data_dir}/{base_data_name}_imgs_merged' if use_merged_dataset else f'{base_path}/{base_data_dir}/{base_data_name}_imgs'
input_annots_dir = f'{base_path}/{base_data_dir}/{base_data_name}_annotations_merged_cleaned' if use_merged_dataset else f'{base_path}/{base_data_dir}/{base_data_name}_annotations'
vis_output_imgs_dir = f'{base_path}/{base_data_dir}/{base_data_name}_vis'
if not os.path.exists(vis_output_imgs_dir):
    os.mkdir(vis_output_imgs_dir)

for i in range(num_folds):
    train_modes = ['train', 'validation']
    for train_mode in train_modes:
        output_imgs_dir = f'{base_path}/{base_data_dir}/{base_data_name}_imgs_merged_{train_mode}_{i}' if use_merged_dataset else f'{base_path}/{base_data_dir}/{base_data_name}_imgs_{train_mode}_{i}'
        output_annots_dir = f'{base_path}/{base_data_dir}/{base_data_name}_annotations_merged_{train_mode}_{i}' if use_merged_dataset else f'{base_path}/{base_data_dir}/{base_data_name}_annotations_{train_mode}_{i}'
        output_masks_dir = f'{base_path}/{base_data_dir}/{base_data_name}_masks_merged_{train_mode}_{i}' if use_merged_dataset else f'{base_path}/{base_data_dir}/{base_data_name}_masks_{train_mode}_{i}'
        if not os.path.exists(output_imgs_dir):
            os.mkdir(output_imgs_dir)
        if not os.path.exists(output_annots_dir):
            os.mkdir(output_annots_dir)
        if generate_masks and not os.path.exists(output_masks_dir):
          os.mkdir(output_masks_dir)

In [3]:
def get_count_type(tiles_dict):
  ids_with_info = []
  for tile in tiles_dict:
    cur_dict = {'id': tile['id'], 'blood_vessel': 0, 'glomerulus': 0, 'unsure': 0}
    for annot in tile['annotations']:
      cur_dict[annot['type']] += 1
    ids_with_info.append(cur_dict)
  return ids_with_info
def calculate_area(coordinates):
    num_points = len(coordinates)
    if num_points < 3:
        return 0

    area = 0
    for i in range(num_points - 1):
        x_i, y_i = coordinates[i]
        x_iplus1, y_iplus1 = coordinates[i + 1]
        area += (x_i * y_iplus1) - (x_iplus1 * y_i)

    x_n, y_n = coordinates[-1]
    x_0, y_0 = coordinates[0]
    area += (x_n * y_0) - (x_0 * y_n)

    area = abs(area / 2)
    return area
# helper function for data visualization
def visualize(**images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(30, 30))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image)
    plt.show()

In [4]:
with open(f'{base_path}/polygons.jsonl', 'r') as json_file:
    json_list = list(json_file)
    
tiles_dicts = []
for json_str in json_list:
    tiles_dicts.append(json.loads(json_str))

In [5]:
tile_df = pd.read_csv(f'{base_path}/tile_meta.csv')
# Find the same stats as the above cell for annotated images
annotated_ids_with_info = get_count_type(tiles_dicts)
annotated_ids_with_info_df = pd.DataFrame.from_dict(annotated_ids_with_info)
tile_df_annotated = pd.merge(tile_df, annotated_ids_with_info_df, on='id', how='inner')
tile_df_annotated.head()

,id,source_wsi,dataset,i,j,blood_vessel,glomerulus,unsure
0,0006ff2aa7cd,2,2,16896,16420,8,1,0
1,00168d1b7522,2,2,14848,14884,1,1,0
2,0033bbc76b6b,1,1,10240,43008,3,0,1
3,003504460b3a,3,2,8192,11776,7,0,0
4,004daf1cbe75,3,2,6144,11264,10,1,0


In [6]:
import math
## Below is the code to create variability by merging adjacent tiles
def get_relevant_annots(tgt_tile_dicts):
  final_tile_dicts = dict()
  for tile_dict in tgt_tile_dicts:
    img_id = tile_dict['id']
    final_annots = []
    for annot in tile_dict['annotations']:
      if annot['type'] == 'blood_vessel':
        final_annots.append(annot['coordinates'][0])
    final_tile_dicts[img_id] = final_annots
  return final_tile_dicts
def get_adjacent_tiles(i, j, img_id, adjacent_tile_dict, dataset_1_pos_dict):
  adjacent_tile_dict[img_id] = dict()
  keys = ['B', 'L', 'R', 'BL', 'BR']
  for k in keys:
    tgt_img_id = None
    if k == 'B':
      pos_to_check = (i, j+512)
    elif k == 'L':
      pos_to_check = (i-512, j)
    elif k == 'R':
      pos_to_check = (i+512, j)
    elif k == 'BL':
      pos_to_check = (i-512, j+512)
    elif k == 'BR':
      pos_to_check = (i+512, j+512)
    if pos_to_check in dataset_1_pos_dict:
      tgt_img_id = dataset_1_pos_dict[pos_to_check]
    adjacent_tile_dict[img_id][k] = tgt_img_id
  return adjacent_tile_dict
def single_shift_polygon(seg_array_1, seg_array_2, shifting_threshold, shift_type, border_eps=1):
    polygon_regions = []
    num_polygons_1 = len(seg_array_1)
    num_polygons_2 = len(seg_array_2)
    
    combined_array_1 = []
    combined_array_2 = []
    for i in range(num_polygons_1):
        polygon_1 = seg_array_1[i]
        new_polygon = []
        for point in polygon_1:
            x, y = point
            if shift_type == 'R':
              if x >= math.ceil(shifting_threshold*512):
                  new_x = x - int(math.ceil(shifting_threshold*512))
                  new_y = y
                  new_polygon.append([new_x, new_y])
            elif shift_type == 'B':
              if y >= math.ceil(shifting_threshold*512):
                new_x = x
                new_y = y - int(math.ceil(shifting_threshold*512))
                new_polygon.append([new_x, new_y])
            elif shift_type == 'L':
              if x <= math.floor((1-shifting_threshold)*512):
                new_x = x + int(math.ceil(shifting_threshold*512))
                new_y = y
                new_polygon.append([new_x, new_y])
        if len(new_polygon) > 0:
          combined_array_1.append(new_polygon)
    
    polygon_regions.append(combined_array_1)
    
    for i in range(num_polygons_2):
        polygon_2 = seg_array_2[i]
        new_polygon = []
        for point in polygon_2:
            x, y = point
            if shift_type == 'R':
              if x <= math.ceil(shifting_threshold*512):
                new_x = x + int(math.floor((1-shifting_threshold)*512))
                new_y = y
                new_polygon.append([new_x, new_y])
            elif shift_type == 'B':
              if y <= math.ceil(shifting_threshold*512):
                new_x = x
                new_y = y + int(math.floor((1-shifting_threshold)*512))
                new_polygon.append([new_x, new_y])
            elif shift_type == 'L':
              if x >= math.floor((1-shifting_threshold)*512):
                  new_x = x - int(math.floor((1-shifting_threshold)*512))
                  new_y = y
                  new_polygon.append([new_x, new_y])
        if len(new_polygon) > 0:
          combined_array_2.append(new_polygon)
    
    polygon_regions.append(combined_array_2)
    
    return polygon_regions

def double_shift_polygon(seg_array_1, seg_array_2, seg_array_3, seg_array_4, shifting_threshold, shift_type, border_eps=1):
  polygon_regions = []
  num_polygons_1 = len(seg_array_1)
  num_polygons_2 = len(seg_array_2)
  num_polygons_3 = len(seg_array_3)
  num_polygons_4 = len(seg_array_4)
  
  combined_array = []
  for i in range(num_polygons_1):
    polygon_1 = seg_array_1[i]
    new_polygon = []
    for point in polygon_1:
        x, y = point
        if shift_type == 'BR':
          if x >= math.ceil(shifting_threshold*512) and y >= math.ceil(shifting_threshold*512):
              new_x = x - int(math.ceil(shifting_threshold*512))
              new_y = y - int(math.ceil(shifting_threshold*512))
              new_polygon.append([new_x, new_y])
        elif shift_type == 'BL':
          if x <= math.floor((1-shifting_threshold)*512) and y >= math.ceil(shifting_threshold*512):
            new_x = x + int(math.ceil(shifting_threshold*512))
            new_y = y - int(math.ceil(shifting_threshold*512))
            new_polygon.append([new_x, new_y])
    if len(new_polygon) > 0:
      combined_array.append(new_polygon)
  
  polygon_regions.append(combined_array)
  
  combined_array = []
  for i in range(num_polygons_2):
    polygon_2 = seg_array_2[i]
    new_polygon = []
    for point in polygon_2:
        x, y = point
        if shift_type == 'BR':
          if x <= math.ceil(shifting_threshold*512) and y >= math.ceil(shifting_threshold*512):
              new_x = x + int(math.floor((1-shifting_threshold)*512))
              new_y = y - int(math.ceil(shifting_threshold*512))
              new_polygon.append([new_x, new_y])
        elif shift_type == 'BL':
          if x >= math.floor((1-shifting_threshold)*512) and y >= math.ceil(shifting_threshold*512):
            new_x = x - int(math.floor((1-shifting_threshold)*512))
            new_y = y - int(math.ceil(shifting_threshold*512))
            new_polygon.append([new_x, new_y])
    if len(new_polygon) > 0:
      combined_array.append(new_polygon)
    
  polygon_regions.append(combined_array)
  
  combined_array = []
  for i in range(num_polygons_3):
    polygon_3 = seg_array_3[i]
    new_polygon = []
    for point in polygon_3:
        x, y = point
        if shift_type == 'BR':
          if x >= math.ceil(shifting_threshold*512) and y <= math.ceil(shifting_threshold*512):
            new_x = x - int(math.ceil(shifting_threshold*512))
            new_y = y + int(math.floor((1-shifting_threshold)*512))
            new_polygon.append([new_x, new_y])
        elif shift_type == 'BL':
          if x <= math.floor((1-shifting_threshold)*512) and y <= math.ceil(shifting_threshold*512):
            new_x = x + int(math.ceil(shifting_threshold*512))
            new_y = y + int(math.floor((1-shifting_threshold)*512))
            new_polygon.append([new_x, new_y])
    if len(new_polygon) > 0:
      combined_array.append(new_polygon)
  
  polygon_regions.append(combined_array)
  
  combined_array = []
  for i in range(num_polygons_4):
    polygon_4 = seg_array_4[i]
    new_polygon = []
    for point in polygon_4:
        x, y = point
        if shift_type == 'BR':
          if x <= math.ceil(shifting_threshold*512) and y <= math.ceil(shifting_threshold*512):
            new_x = x + int(math.floor((1-shifting_threshold)*512))
            new_y = y + int(math.floor((1-shifting_threshold)*512))
            new_polygon.append([new_x, new_y])
        elif shift_type == 'BL':
          if x >= math.floor((1-shifting_threshold)*512) and y <= math.ceil(shifting_threshold*512):
            new_x = x - int(math.floor((1-shifting_threshold)*512))
            new_y = y + int(math.floor((1-shifting_threshold)*512))
            new_polygon.append([new_x, new_y])
    if len(new_polygon) > 0:
      combined_array.append(new_polygon)
  
  polygon_regions.append(combined_array)
  
  return polygon_regions

def check_polygon_border(pt, border_eps=1):
  return pt[0] <= border_eps or pt[0] >= 512-border_eps or pt[1] <= border_eps or pt[1] >= 512-border_eps

def single_shift_img(img_1, img_2, shifting_threshold, shift_type):
    height, width, _ = img_1.shape
    
    if shift_type == 'R':
      img_1_part = img_1[:, int(math.ceil(shifting_threshold*width)):, :]
      img_2_part = img_2[:, :int(math.ceil(shifting_threshold*width)), :]
      combined_image = cv2.hconcat([img_1_part, img_2_part])
    elif shift_type == 'B':
      img_1_part = img_1[int(math.ceil(shifting_threshold*height)):, :, :]
      img_2_part = img_2[:int(math.ceil(shifting_threshold*height)), :, :]
      combined_image = cv2.vconcat([img_1_part, img_2_part])
    else:
      img_1_part = img_1[:, :int(math.floor((1-shifting_threshold)*width)), :]
      img_2_part = img_2[:, int(math.floor((1-shifting_threshold)*width)):, :]
      combined_image = cv2.hconcat([img_2_part, img_1_part])
    return combined_image

def double_shift_img(img_1, img_2, img_3, img_4, shifting_threshold, shift_type):
  height, width, _ = img_1.shape
  if shift_type == 'BR':
    img_1_part = img_1[int(math.ceil(shifting_threshold*height)):, int(math.ceil(shifting_threshold*width)):, :]
    img_2_part = img_2[int(math.ceil(shifting_threshold*height)):, :int(math.ceil(shifting_threshold*width)), :]
    img_3_part = img_3[:int(math.ceil(shifting_threshold*height)), int(math.ceil(shifting_threshold*width)):, :]
    img_4_part = img_4[:int(math.ceil(shifting_threshold*height)), :int(math.ceil(shifting_threshold*width)), :]
    image_top = cv2.hconcat([img_1_part, img_2_part])
    image_bottom = cv2.hconcat([img_3_part, img_4_part])
    combined_image = cv2.vconcat([image_top, image_bottom])
  else:
    img_1_part = img_1[int(math.ceil(shifting_threshold*height)):, :int(math.floor((1-shifting_threshold)*width)), :]
    img_2_part = img_2[int(math.ceil(shifting_threshold*height)):, int(math.floor((1-shifting_threshold)*width)):, :]
    img_3_part = img_3[:int(math.ceil(shifting_threshold*height)), :int(math.floor((1-shifting_threshold)*width)), :]
    img_4_part = img_4[:int(math.ceil(shifting_threshold*height)), int(math.floor((1-shifting_threshold)*width)):, :]
    image_top = cv2.hconcat([img_2_part, img_1_part])
    image_bottom = cv2.hconcat([img_4_part, img_3_part])
    combined_image = cv2.vconcat([image_top, image_bottom])
  return combined_image

def load_img(img_dir, img_id):
  img = cv2.imread(f"{img_dir}/{img_id}.png")
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img

def add_border(image, border_size, border_color):
    height, width = image.shape[:2]
    new_height = height + 2 * border_size
    new_width = width + 2 * border_size
    bordered_image = np.zeros((new_height, new_width, 3), dtype=np.uint8)
    bordered_image[border_size:height + border_size, border_size:width + border_size] = image
    cv2.rectangle(bordered_image, (0, 0), (new_width - 1, new_height - 1), border_color, border_size)
    return bordered_image

def create_coco_annots(all_coords, pair_format=True):
  cur_tile_coco_annots = []
  for coords in all_coords:
#     print('======================')
#     print(coords)
#     print('======================')
    if not pair_format:
      coords = [[pt[0], pt[1]] for pt in zip(coords[::2], coords[1::2])]
    segmentations = [[pt for pair in coords for pt in pair]]
    segmentation_area = calculate_area(coords)
    min_x = min(coords, key=lambda x: x[0])[0]
    max_x = max(coords, key=lambda x: x[0])[0]
    min_y = min(coords, key=lambda x: x[1])[1]
    max_y = max(coords, key=lambda x: x[1])[1]
    segmentation_bbox = [min_x, min_y, max_x-min_x, max_y-min_y]
    category_id = 0
    cur_tile_coco_annots.append({
      'segmentation': segmentations,
      'area': segmentation_area,
      'bbox': segmentation_bbox,
      'category_id': category_id
    })
  return cur_tile_coco_annots

def save_merged_imgs(input_imgs_dir, output_imgs_dir, base_img_id, base_img_dict, shifting_thresholds):
  img_arr_dict = {'base_img': load_img(input_imgs_dir, base_img_id)}
  cv2.imwrite(f'{output_imgs_dir}/{base_img_id}.png', img_arr_dict['base_img'])
  for shift_type in base_img_dict.keys():
    if base_img_dict[shift_type] is not None:
      img_arr_dict[shift_type] = load_img(input_imgs_dir, base_img_dict[shift_type])
    else:
      img_arr_dict[shift_type] = None
  for shifting_threshold in shifting_thresholds:
    for shift_type in img_arr_dict:
      if img_arr_dict[shift_type] is not None:
        if shift_type in ['R', 'B']:
          shifted_img = single_shift_img(img_arr_dict['base_img'], img_arr_dict[shift_type], shifting_threshold, shift_type)
          if shifted_img.shape[0] != 512 or shifted_img.shape[1] != 512 or shifted_img.shape[2] != 3:
            print(f'Error!!! Got an odd combined shape: {shifted_img.shape} for image id: {base_img_id} and shift type: {shift_type} and shifting threshold: {shifting_threshold}')
          cv2.imwrite(f'{output_imgs_dir}/{base_img_id}_{shift_type}_{shifting_threshold}.png', shifted_img)
        elif shift_type == 'BR' and img_arr_dict['B'] is not None and img_arr_dict['R'] is not None:
          shifted_img = double_shift_img(img_arr_dict['base_img'], img_arr_dict['R'], img_arr_dict['B'], img_arr_dict[shift_type], shifting_threshold, shift_type)
          if shifted_img.shape[0] != 512 or shifted_img.shape[1] != 512 or shifted_img.shape[2] != 3:
            print(f'Error!!! Got an odd combined shape: {shifted_img.shape} for image id: {base_img_id} and shift type: {shift_type}')
          cv2.imwrite(f'{output_imgs_dir}/{base_img_id}_{shift_type}_{shifting_threshold}.png', shifted_img)
        elif shift_type == 'BL' and img_arr_dict['B'] is not None and img_arr_dict['L'] is not None:
          shifted_img = double_shift_img(img_arr_dict['base_img'], img_arr_dict['L'], img_arr_dict['B'], img_arr_dict[shift_type], shifting_threshold, shift_type)
          if shifted_img.shape[0] != 512 or shifted_img.shape[1] != 512 or shifted_img.shape[2] != 3:
            print(f'Error!!! Got an odd combined shape: {shifted_img.shape} for image id: {base_img_id} and shift type: {shift_type}')
          cv2.imwrite(f'{output_imgs_dir}/{base_img_id}_{shift_type}_{shifting_threshold}.png', shifted_img)

def save_merged_annotations(output_annots_dir, base_img_id, base_img_dict, shifting_thresholds, tgt_tile_dicts):
  print(f'base image id: {base_img_id}')
  cur_tile_annots = create_coco_annots(tgt_tile_dicts[base_img_id])
  with open(f'{output_annots_dir}/{base_img_id}.pkl', 'wb') as f:
    pickle.dump(cur_tile_annots, f, protocol=pickle.HIGHEST_PROTOCOL)
  for shifting_threshold in shifting_thresholds:
    for shift_type in base_img_dict:
      if base_img_dict[shift_type] is not None:
        if shift_type in ['R', 'B']:
          polygon_regions = single_shift_polygon(tgt_tile_dicts[base_img_id], tgt_tile_dicts[base_img_dict[shift_type]], shifting_threshold, shift_type)
#           print(f'===========after single shift - {len(polygon_regions)}============')
#           print(polygon_regions)
#           print('===========after single shift============')
          shifted_coords = get_polygon_merged_coords(polygon_regions)
#           print(f'===========after polyon merge - {len(shifted_coords)}============')
#           print(shifted_coords)
#           print('===========after polyon merge============')
#           print(f'===========merged coords shift type R B===========')
#           print(shifted_coords)
#           print(f'===========merged coords shift type R B===========')
          shifted_annots = create_coco_annots(shifted_coords, pair_format=False)
#           print(f'===========after coco - {len(shifted_coords)}============')
#           print(shifted_annots)
#           print('===========after coco============')
          with open(f'{output_annots_dir}/{base_img_id}_{shift_type}_{shifting_threshold}.pkl', 'wb') as f:
            pickle.dump(shifted_annots, f, protocol=pickle.HIGHEST_PROTOCOL)
        elif shift_type == 'BR' and base_img_dict['B'] is not None and base_img_dict['R'] is not None:
          polygon_regions = double_shift_polygon(tgt_tile_dicts[base_img_id], tgt_tile_dicts[base_img_dict['R']], tgt_tile_dicts[base_img_dict['B']], tgt_tile_dicts[base_img_dict[shift_type]], shifting_threshold, shift_type)
          shifted_coords = get_polygon_merged_coords(polygon_regions)
#           print(f'===========merged coords shift type BR===========')
#           print(shifted_coords)
#           print(f'===========merged coords shift type BR===========')
          shifted_annots = create_coco_annots(shifted_coords, pair_format=False)
          with open(f'{output_annots_dir}/{base_img_id}_{shift_type}_{shifting_threshold}.pkl', 'wb') as f:
            pickle.dump(shifted_annots, f, protocol=pickle.HIGHEST_PROTOCOL)
        elif shift_type == 'BL' and base_img_dict['B'] is not None and base_img_dict['L'] is not None:
          polygon_regions = double_shift_polygon(tgt_tile_dicts[base_img_id], tgt_tile_dicts[base_img_dict['L']], tgt_tile_dicts[base_img_dict['B']], tgt_tile_dicts[base_img_dict[shift_type]], shifting_threshold, shift_type)
          shifted_coords = get_polygon_merged_coords(polygon_regions)
#           print(f'===========merged coords shift type BL===========')
#           print(shifted_coords)
#           print(f'===========merged coords shift type BL===========')
          shifted_annots = create_coco_annots(shifted_coords, pair_format=False)
          with open(f'{output_annots_dir}/{base_img_id}_{shift_type}_{shifting_threshold}.pkl', 'wb') as f:
            pickle.dump(shifted_annots, f, protocol=pickle.HIGHEST_PROTOCOL)

def get_annotated_img(imgs_dir, annots_dir, base_img_id):
  eps = 1
  annotated_image_coco = load_img(imgs_dir, base_img_id)
  img_height, img_width, _ = annotated_image_coco.shape
  with open(f'{annots_dir}/{base_img_id}.pkl', 'rb') as f:
    tgt_annots = pickle.load(f)
  for tgt_annot in tgt_annots:
    coords = [[x, y] for x, y in zip(tgt_annot['segmentation'][0][::2], tgt_annot['segmentation'][0][1::2])]
    min_x, min_y, width, height = tgt_annot['bbox']
    cv2.fillPoly(annotated_image_coco, pts=[np.array(coords)], color=(0,255,0))
    top_left = (int(tgt_annot['bbox'][0]), int(tgt_annot['bbox'][1]))
    min_x = min(coords, key=lambda x: x[0])[0]
    min_y = min(coords, key=lambda x: x[1])[1]
    max_x = max(coords, key=lambda x: x[0])[0]
    max_y = max(coords, key=lambda x: x[1])[1]
    bottom_right = (int(tgt_annot['bbox'][0]+tgt_annot['bbox'][2]), int(tgt_annot['bbox'][1]+tgt_annot['bbox'][3]))
    print(max_x-min_x, max_y-min_y, tgt_annot['bbox'])
#     if min_x <= eps or max_x >= img_width-eps or min_y <= eps or max_y >= img_height-eps:
#       annotated_image_coco = cv2.rectangle(annotated_image_coco, top_left, bottom_right, (255,0,0), 2)
    annotated_image_coco = cv2.rectangle(annotated_image_coco, top_left, bottom_right, (255,0,0), 2)
  annotated_image_coco = add_border(annotated_image_coco, 5, (0,0,255))
  return annotated_image_coco

def get_polygon_merged_annotated_img(imgs_dir, annots_dir, base_img_id):
  eps = 1
  annotated_image_coco = load_img(imgs_dir, base_img_id)
  img_height, img_width, _ = annotated_image_coco.shape
  annotated_image_mask = np.zeros((img_height,img_width))
  with open(f'{annots_dir}/{base_img_id}.pkl', 'rb') as f:
    tgt_annots = pickle.load(f)
  all_polygons = [x for tgt_annot in tgt_annots for x in tgt_annot['segmentation']]
#   all_polygons = merge_connected_polygons(all_polygons, 1)
  for polygon in all_polygons:
    coords = [[x, y] for x, y in zip(polygon[::2], polygon[1::2])]
    cv2.fillPoly(annotated_image_mask, pts=[np.array(coords)], color=255)
  merged_polygons = get_segmentation_coordinates(annotated_image_mask)
  for merged_polygon in merged_polygons:
    coords = [[x, y] for x, y in zip(merged_polygon[::2], merged_polygon[1::2])]
    cv2.fillPoly(annotated_image_coco, pts=[np.array(coords)], color=(0,255,0))
    min_x = min(coords, key=lambda x: x[0])[0]
    min_y = min(coords, key=lambda x: x[1])[1]
    max_x = max(coords, key=lambda x: x[0])[0]
    max_y = max(coords, key=lambda x: x[1])[1]
    top_left = (int(min_x), int(min_y))
    bottom_right = (int(max_x), int(max_y))
#     if min_x <= eps or max_x >= img_width-eps or min_y <= eps or max_y >= img_height-eps:
#       annotated_image_coco = cv2.rectangle(annotated_image_coco, top_left, bottom_right, (255,0,0), 2)
    annotated_image_coco = cv2.rectangle(annotated_image_coco, top_left, bottom_right, (255,0,0), 2)
  annotated_image_coco = add_border(annotated_image_coco, 5, (0,0,255))
  return annotated_image_coco

def get_polygon_merged_coords(polygons_regions, epsilon=1):
  merged_polygons = group_merged_polygons(polygons_regions, epsilon=epsilon)
  final_segmentation_coords = []
  
  for merged_polygon in merged_polygons:
    annotated_image_mask = np.zeros((512,512))
    for coords in merged_polygon:
      cv2.fillPoly(annotated_image_mask, pts=[np.array(coords)], color=255)
#     plt.imshow(annotated_image_mask)
#     plt.show()
    final_segmentation_coords += get_segmentation_coordinates(annotated_image_mask)
  
  return final_segmentation_coords
  
import math

def group_merged_polygons(regions, epsilon):
    merged_groups = []
    
    def merge_polygons(poly1, poly2):
        return poly1 + poly2
    
    def distance(point1, point2):
        return math.sqrt((point2[0] - point1[0]) ** 2 + (point2[1] - point1[1]) ** 2)
    
    def are_polygons_close(poly1, poly2):
        for p1 in poly1:
            for p2 in poly2:
                if distance(p1, p2) <= epsilon:
                    return True
        return False
    
    for region_index, region in enumerate(regions):
        for polygon in region:
            merged = False
            for group in merged_groups:
                if (not any(region_index == group_region_idx for group_region_idx, _ in group)) and any(are_polygons_close(polygon, poly) for _, poly in group):
                    group.append([region_index, polygon])
                    merged = True
                    break
            if not merged:
                merged_groups.append([[region_index, polygon]])
    
    merged_polygons = []
    for group in merged_groups:
        polygon_group = []
        for _, polygon in group:
          polygon_group.append(polygon)
        merged_polygons.append(polygon_group)
    return merged_polygons


def get_segmentation_coordinates(mask):
    mask = np.uint8(mask)
    
    # Find contours in the binary mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Extract segmentation coordinates from the contours
    segmentation_coords = []
    for contour in contours:
        contour = np.squeeze(contour)
        coords = contour.flatten().tolist()
        segmentation_coords.append(coords)

    return segmentation_coords

def save_merged_img_id_with_annot(output_imgs_dir, output_annots_dir, vis_output_imgs_dir, base_img_id, base_img_dict, shifting_thresholds):
  img_arr_dict = {'base_image': get_annotated_img(output_imgs_dir, output_annots_dir, base_img_id)}
  for shift_type in base_img_dict.keys():
    if base_img_dict[shift_type] is not None:
      img_arr_dict[shift_type] = get_annotated_img(output_imgs_dir, output_annots_dir, base_img_dict[shift_type])
    else:
      img_arr_dict[shift_type] = add_border((np.zeros((512, 512, 3), dtype="uint8")), 5, (0,0,255))
  image_top = cv2.hconcat([img_arr_dict['L'], img_arr_dict['base_image'], img_arr_dict['R']])
  print(img_arr_dict['BL'].shape, img_arr_dict['B'].shape, img_arr_dict['BR'].shape)
  image_bottom = cv2.hconcat([img_arr_dict['BL'], img_arr_dict['B'], img_arr_dict['BR']])
  image_full = cv2.vconcat([image_top, image_bottom])
  cv2.imwrite(f'{vis_output_imgs_dir}/{base_img_id}_full_annotated.png', image_full)
  all_merged_img_ids = [x.split('.png')[0] for x in os.listdir(output_imgs_dir) if base_img_id in x and x != f'{base_img_id}.png']
  for merged_img_id in all_merged_img_ids:
    merged_annotated_img = get_annotated_img(output_imgs_dir, output_annots_dir, merged_img_id)
    cv2.imwrite(f'{vis_output_imgs_dir}/{merged_img_id}_full_annotated.png', merged_annotated_img)
#     polygon_merged_annotated_img = get_polygon_merged_annotated_img(output_imgs_dir, output_annots_dir, merged_img_id)
#     cv2.imwrite(f'{vis_output_imgs_dir}/{merged_img_id}_full_annotated_polygon_merged.png', polygon_merged_annotated_img)

In [7]:
# import random
# random.seed(42)
# all_dataset_tile_ids = sorted(list(tile_df_annotated.loc[:, 'id'].values))
# dataset_1 = tile_df_annotated.loc[tile_df_annotated['dataset']==1]
# dataset_1_tile_ids = sorted(list(dataset_1.loc[:, 'id'].values))
# wsi_1_ids = list(dataset_1.loc[dataset_1['source_wsi']==1, 'id'])
# wsi_2_ids = list(dataset_1.loc[dataset_1['source_wsi']==2, 'id'])
# tgt_tile_dicts = [x for x in tiles_dicts if x['id'] in dataset_1_tile_ids]
# tgt_tile_dicts = get_relevant_annots(tgt_tile_dicts)
# print(len(all_dataset_tile_ids), len(dataset_1_tile_ids))
# random.shuffle(dataset_1_tile_ids)
# random.shuffle(wsi_1_ids)
# random.shuffle(wsi_2_ids)

In [8]:
import random
random.seed(42)
dataset_1_tile_ids = sorted(list(tile_df_annotated.loc[tile_df_annotated['dataset']==1, 'id'].values))
all_dataset_tile_ids = sorted(list(tile_df_annotated.loc[:, 'id'].values))
tgt_wsis = [1,2,3,4]
wsi_tile_ids = []
for tgt_wsi in tgt_wsis:
    if generate_all_datset_annots:
        wsi_ids = list(tile_df_annotated.loc[tile_df_annotated['source_wsi']==tgt_wsi, 'id'].values)
    else:
        wsi_ids = list(tile_df_annotated.loc[(tile_df_annotated['dataset']==1) & (tile_df_annotated['source_wsi']==tgt_wsi), 'id'].values)
    random.shuffle(wsi_ids)
    wsi_tile_ids.append(wsi_ids)

In [9]:
wsi_tile_ids[0]

['cfb0bd617652',
 '678c51bde237',
 '2519a0f1b867',
 '523a3912a7ee',
 '9a70c2d9e4f4',
 'ca081412bad8',
 'd9547a630fab',
 '5384b927f5b3',
 '2d2093e38e39',
 '3e248be7dd4c',
 '39b8aafd630b',
 '578c8b32057e',
 '4c058755d597',
 '23c30a541c7a',
 '9f999c74b67e',
 '0e8aed930dc6',
 '03d335057db3',
 '328a651a4ca7',
 '55b07829e888',
 '4cf2563142f4',
 '6d1315ab9ef7',
 '7483bb77d145',
 'e506a133c873',
 '18be061202ea',
 'f1af97caff9d',
 'db8adbc51c82',
 '13aa34ced90d',
 '8f6f3e18eda5',
 '033a656390b2',
 '5ed62676da45',
 '76e5b8a1caa1',
 '415a900f7a89',
 '9ed941a933d0',
 'ac68833641fe',
 '1c0295c1d01d',
 '4a1b99c7ab16',
 'f6b0c55c34c7',
 '15ed954c0cdf',
 '0a993633aa5e',
 'd6529e6e8602',
 '242db333c452',
 '8e90e6189c6b',
 '783e8f735476',
 '4b197545ea0e',
 '45d6f53ececf',
 '5a89c7f7ad05',
 '9b95807e6b23',
 '30ad648e36a2',
 'cb96f7b681f8',
 '4e8b6849684d',
 '303b72a38e29',
 '41225437e33e',
 'ac8f685801d0',
 'abbacf024cb5',
 '95c077870f91',
 'dcea4e9d6726',
 '7392e21e0fe2',
 'e1647f94f1f9',
 '1a54cda8f32d

In [ ]:
len(all_dataset_tile_ids), len(wsi_tile_ids)

In [ ]:
input_imgs_dir, input_annots_dir

In [ ]:
fold_size = int(math.ceil(len(dataset_1_tile_ids) / num_folds))
print(f'Fold size is {fold_size}')
shifting_thresholds = [0.15,0.30,0.45,0.60,0.75,0.90]
for i in range(1):
  validation_imgs_dir = f'{base_path}/{base_data_dir}/{base_data_name}_imgs_merged_validation_{i}' if use_merged_dataset else f'{base_path}/{base_data_dir}/{base_data_name}_imgs_validation_{i}'
  validation_annots_dir = f'{base_path}/{base_data_dir}/{base_data_name}_annotations_merged_validation_{i}' if use_merged_dataset else f'{base_path}/{base_data_dir}/{base_data_name}_annotations_validation_{i}'
  train_imgs_dir = f'{base_path}/{base_data_dir}/{base_data_name}_imgs_merged_train_{i}' if use_merged_dataset else f'{base_path}/{base_data_dir}/{base_data_name}_imgs_train_{i}'
  train_annots_dir = f'{base_path}/{base_data_dir}/{base_data_name}_annotations_merged_train_{i}' if use_merged_dataset else f'{base_path}/{base_data_dir}/{base_data_name}_annotations_train_{i}'
  
  if pick_validation_from_file:
    with open(f'{base_path}/dataset1_files/validation_wsi_1_fold_img_ids.pkl', 'rb') as f:
        wsi_1_validation_img_ids = pickle.load(f)
    with open(f'{base_path}/dataset1_files/validation_wsi_2_fold_img_ids.pkl', 'rb') as f:
        wsi_2_validation_img_ids = pickle.load(f)
    wsi_validation_img_ids = wsi_1_validation_img_ids[i] + wsi_2_validation_img_ids[i]
    cur_validaton_slice = wsi_validation_img_ids
    cur_training_slice = list(set(dataset_1_tile_ids) - set(cur_validaton_slice))
    print(len(cur_validaton_slice), len(cur_training_slice))
    print(cur_validaton_slice, cur_training_slice)
  else:
    if get_whole_dataset_into_fold:
        cur_validaton_slice = wsi_1_ids + wsi_2_ids
        cur_training_slice = wsi_1_ids + wsi_2_ids
    else:
#         cur_validaton_slice = wsi_1_ids[int(i*(fold_size/2)):min(int((i+1)*(fold_size/2)), len(wsi_1_ids))] + wsi_2_ids[int(i*(fold_size/2)):min(int((i+1)*(fold_size/2)), len(wsi_2_ids))]
#         cur_training_slice = list(set(dataset_1_tile_ids) - set(cur_validaton_slice)) if not generate_all_datset_annots else list(set(all_dataset_tile_ids) - set(cur_validaton_slice))
        # Temporary changes below
        cur_validaton_slice = []
        num_wsis = float(len(wsi_tile_ids))
        for wsi_tile_id in wsi_tile_ids:
            cur_validaton_slice += wsi_tile_id[int(i*(fold_size/num_wsis)):min(int((i+1)*(fold_size/num_wsis)), len(wsi_tile_id))]
        cur_training_slice = list(set(dataset_1_tile_ids) - set(cur_validaton_slice)) if not generate_all_datset_annots else list(set(all_dataset_tile_ids) - set(cur_validaton_slice))
  
  print('Saving validation images and annotations...')
  for validation_img_id in cur_validaton_slice:
      shutil.copy(f'{input_imgs_dir}/{validation_img_id}.png', f'{validation_imgs_dir}/{validation_img_id}.png')
      shutil.copy(f'{input_annots_dir}/{validation_img_id}.pkl', f'{validation_annots_dir}/{validation_img_id}.pkl')
    
  print('Saving training images and annotations...')
  for train_img_id in cur_training_slice:
    shutil.copy(f'{input_imgs_dir}/{train_img_id}.png', f'{train_imgs_dir}/{train_img_id}.png')
    shutil.copy(f'{input_annots_dir}/{train_img_id}.pkl', f'{train_annots_dir}/{train_img_id}.pkl')

In [ ]:
i=0
validation_annots_dir = f'{base_path}/{base_data_dir}/{base_data_name}_annotations_merged_validation_{i}' if use_merged_dataset else f'{base_path}/{base_data_dir}/{base_data_name}_annotations_validation_{i}'
files = os.listdir(validation_annots_dir)
wsi_counts = [0]*len(wsi_tile_ids)
for f in files:
    img_id = f.split('.pkl')[0]
    for idx, wsi_tile_id in enumerate(wsi_tile_ids):
        if img_id in wsi_tile_id:
            wsi_counts[idx] += 1
wsi_counts

In [ ]:
fold_size = int(math.ceil(len(dataset_1_tile_ids) / num_folds))
print(f'Fold size is {fold_size}')
shifting_thresholds = [0.15,0.30,0.45,0.60,0.75,0.90]
print(f'Fold size: {fold_size}')
for i in range(1):
  validation_imgs_dir = f'{base_path}/{base_data_dir}/{base_data_name}_imgs_merged_validation_{i}' if use_merged_dataset else f'{base_path}/{base_data_dir}/{base_data_name}_imgs_validation_{i}'
  validation_annots_dir = f'{base_path}/{base_data_dir}/{base_data_name}_annotations_merged_validation_{i}' if use_merged_dataset else f'{base_path}/{base_data_dir}/{base_data_name}_annotations_validation_{i}'
  train_imgs_dir = f'{base_path}/{base_data_dir}/{base_data_name}_imgs_merged_train_{i}' if use_merged_dataset else f'{base_path}/{base_data_dir}/{base_data_name}_imgs_train_{i}'
  train_annots_dir = f'{base_path}/{base_data_dir}/{base_data_name}_annotations_merged_train_{i}' if use_merged_dataset else f'{base_path}/{base_data_dir}/{base_data_name}_annotations_train_{i}'
  
  if pick_validation_from_file:
    with open(f'{base_path}/dataset1_files/validation_wsi_1_fold_img_ids.pkl', 'rb') as f:
        wsi_1_validation_img_ids = pickle.load(f)
    with open(f'{base_path}/dataset1_files/validation_wsi_2_fold_img_ids.pkl', 'rb') as f:
        wsi_2_validation_img_ids = pickle.load(f)
    wsi_validation_img_ids = wsi_1_validation_img_ids[i] + wsi_2_validation_img_ids[i]
    cur_validaton_slice = wsi_validation_img_ids
    cur_training_slice = list(set(dataset_1_tile_ids) - set(cur_validaton_slice))
    print(len(cur_validaton_slice), len(cur_training_slice))
    print(cur_validaton_slice, cur_training_slice)
  else:
#     cur_validaton_slice = dataset_1_tile_ids[int(i*fold_size):min(int((i+1)*fold_size), len(dataset_1_tile_ids))]
#     cur_training_slice = dataset_1_tile_ids[:int(i*fold_size)] + dataset_1_tile_ids[min(int((i+1)*fold_size), len(dataset_1_tile_ids)):]
    if get_whole_dataset_into_fold:
        cur_validaton_slice = wsi_1_ids + wsi_2_ids
        cur_training_slice = wsi_1_ids + wsi_2_ids
    else:
#         cur_validaton_slice = wsi_1_ids[int(i*(fold_size/2)):min(int((i+1)*(fold_size/2)), len(wsi_1_ids))] + wsi_2_ids[int(i*(fold_size/2)):min(int((i+1)*(fold_size/2)), len(wsi_2_ids))]
#         cur_training_slice = list(set(dataset_1_tile_ids) - set(cur_validaton_slice)) if not generate_all_datset_annots else list(set(all_dataset_tile_ids) - set(cur_validaton_slice))
        # Temporary changes below
        cur_validaton_slice = []
        num_wsis = float(len(wsi_tile_ids))
        for wsi_tile_id in wsi_tile_ids:
            cur_validaton_slice += wsi_tile_id[int(i*(fold_size/num_wsis)):min(int((i+1)*(fold_size/num_wsis)), len(wsi_tile_id))]
        cur_training_slice = list(set(dataset_1_tile_ids) - set(cur_validaton_slice)) if not generate_all_datset_annots else list(set(all_dataset_tile_ids) - set(cur_validaton_slice))
  
  print('Saving validation images and annotations...')
  for validation_img_id in cur_validaton_slice:
      shutil.copy(f'{input_imgs_dir}/{validation_img_id}.png', f'{validation_imgs_dir}/{validation_img_id}.png')
      shutil.copy(f'{input_annots_dir}/{validation_img_id}.pkl', f'{validation_annots_dir}/{validation_img_id}.pkl')
  
#   print('Creating adjacent tile dict for the current validation set...')
#   dataset_1_wsi_1_pos_dict = {(x[0], x[1]): x[-1] for x in dataset_1.loc[(dataset_1['source_wsi']==1) & (dataset_1['id'].isin(cur_validaton_slice)), ['i', 'j', 'id']].values}
#   dataset_1_wsi_2_pos_dict = {(x[0], x[1]): x[-1] for x in dataset_1.loc[(dataset_1['source_wsi']==2) & (dataset_1['id'].isin(cur_validaton_slice)), ['i', 'j', 'id']].values}
#   reverse_dataset_1_wsi_1_pos_dict = {v:k for k,v in dataset_1_wsi_1_pos_dict.items()}
#   reverse_dataset_1_wsi_2_pos_dict = {v:k for k,v in dataset_1_wsi_2_pos_dict.items()}
#   adjacent_tile_dict = dict()
#   for i,j in dataset_1_wsi_1_pos_dict.keys():
#     adjacent_tile_dict = get_adjacent_tiles(i, j, dataset_1_wsi_1_pos_dict[(i,j)], adjacent_tile_dict, dataset_1_wsi_1_pos_dict)
#   for i,j in dataset_1_wsi_2_pos_dict.keys():
#     adjacent_tile_dict = get_adjacent_tiles(i, j, dataset_1_wsi_2_pos_dict[(i,j)], adjacent_tile_dict, dataset_1_wsi_2_pos_dict)
  
#   print('Saving validation images and annotations...')
#   for validation_img_id in adjacent_tile_dict.keys():
#     shutil.copy(f'{input_imgs_dir}/{validation_img_id}.png', f'{validation_imgs_dir}/{validation_img_id}.png')
#     shutil.copy(f'{input_annots_dir}/{validation_img_id}.pkl', f'{validation_annots_dir}/{validation_img_id}.pkl')
#     validation_adjacent_tiles = adjacent_tile_dict[validation_img_id]
#     valid_shifts = []
#     if validation_adjacent_tiles['R'] is not None:
#       valid_shifts.append('R')
#     if validation_adjacent_tiles['B'] is not None:
#       valid_shifts.append('B')
#     if validation_adjacent_tiles['B'] is not None and validation_adjacent_tiles['R'] is not None and validation_adjacent_tiles['BR'] is not None:
#       valid_shifts.append('BR')
#     if validation_adjacent_tiles['B'] is not None and validation_adjacent_tiles['L'] is not None and validation_adjacent_tiles['BL'] is not None:
#       valid_shifts.append('BL')
#     for valid_shift in valid_shifts:
#       for shifting_threshold in shifting_thresholds:
#         shutil.copy(f'{input_imgs_dir}/{validation_img_id}_{valid_shift}_{shifting_threshold}.png', f'{validation_imgs_dir}/{validation_img_id}_{valid_shift}_{shifting_threshold}.png')
#         shutil.copy(f'{input_annots_dir}/{validation_img_id}_{valid_shift}_{shifting_threshold}.pkl', f'{validation_annots_dir}/{validation_img_id}_{valid_shift}_{shifting_threshold}.pkl')
  
  print('Creating adjacent tile dict for the current training set...')
  dataset_1_wsi_1_pos_dict = {(x[0], x[1]): x[-1] for x in dataset_1.loc[(dataset_1['source_wsi']==1) & (dataset_1['id'].isin(cur_training_slice)), ['i', 'j', 'id']].values}
  dataset_1_wsi_2_pos_dict = {(x[0], x[1]): x[-1] for x in dataset_1.loc[(dataset_1['source_wsi']==2) & (dataset_1['id'].isin(cur_training_slice)), ['i', 'j', 'id']].values}
  reverse_dataset_1_wsi_1_pos_dict = {v:k for k,v in dataset_1_wsi_1_pos_dict.items()}
  reverse_dataset_1_wsi_2_pos_dict = {v:k for k,v in dataset_1_wsi_2_pos_dict.items()}
  adjacent_tile_dict = dict()
  for i,j in dataset_1_wsi_1_pos_dict.keys():
    adjacent_tile_dict = get_adjacent_tiles(i, j, dataset_1_wsi_1_pos_dict[(i,j)], adjacent_tile_dict, dataset_1_wsi_1_pos_dict)
  for i,j in dataset_1_wsi_2_pos_dict.keys():
    adjacent_tile_dict = get_adjacent_tiles(i, j, dataset_1_wsi_2_pos_dict[(i,j)], adjacent_tile_dict, dataset_1_wsi_2_pos_dict)
  
  print('Saving training images and annotations...')
  for train_img_id in cur_training_slice:
    shutil.copy(f'{input_imgs_dir}/{train_img_id}.png', f'{train_imgs_dir}/{train_img_id}.png')
    if should_dilate:
        with open(f'{input_annots_dir}/{train_img_id}.pkl', 'rb') as f:
            annotations = pickle.load(f)
        cur_tile_coco_annots = []
        for annot in annotations:
            coords = [[x,y] for x,y in zip(annot['segmentation'][0][::2], annot['segmentation'][0][1::2])]
            mask = np.zeros((512,512), dtype=np.uint8)
            cv2.fillPoly(mask, pts=[np.array(coords)], color=1)
            kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
            dilated_mask = cv2.dilate(mask, kernel, iterations=2)
            contours, _ = cv2.findContours(dilated_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            dilated_coords = contours[0].reshape(-1, 2)
            coords = dilated_coords
            segmentations = [[pt for pair in coords for pt in pair]]
            segmentation_area = calculate_area(coords)
            min_x = min(coords, key=lambda x: x[0])[0]
            max_x = max(coords, key=lambda x: x[0])[0]
            min_y = min(coords, key=lambda x: x[1])[1]
            max_y = max(coords, key=lambda x: x[1])[1]
            segmentation_bbox = [min_x, min_y, max_x-min_x, max_y-min_y]
            category_id = annot['category_id']
            cur_tile_coco_annots.append({
              'segmentation': segmentations,
              'area': segmentation_area,
              'bbox': segmentation_bbox,
              'category_id': category_id
            })
        with open(f'{train_annots_dir}/{train_img_id}.pkl', 'wb') as f:
            pickle.dump(cur_tile_coco_annots, f, protocol=pickle.HIGHEST_PROTOCOL)
    else:
        shutil.copy(f'{input_annots_dir}/{train_img_id}.pkl', f'{train_annots_dir}/{train_img_id}.pkl')
    if use_merged_dataset:
        train_adjacent_tiles = adjacent_tile_dict[train_img_id]
        valid_shifts = []
        if train_adjacent_tiles['R'] is not None:
          valid_shifts.append('R')
        if train_adjacent_tiles['B'] is not None:
          valid_shifts.append('B')
        if train_adjacent_tiles['B'] is not None and train_adjacent_tiles['R'] is not None and train_adjacent_tiles['BR'] is not None:
          valid_shifts.append('BR')
        if train_adjacent_tiles['B'] is not None and train_adjacent_tiles['L'] is not None and train_adjacent_tiles['BL'] is not None:
          valid_shifts.append('BL')
        for valid_shift in valid_shifts:
          for shifting_threshold in shifting_thresholds:
            shutil.copy(f'{input_imgs_dir}/{train_img_id}_{valid_shift}_{shifting_threshold}.png', f'{train_imgs_dir}/{train_img_id}_{valid_shift}_{shifting_threshold}.png')
            shutil.copy(f'{input_annots_dir}/{train_img_id}_{valid_shift}_{shifting_threshold}.pkl', f'{train_annots_dir}/{train_img_id}_{valid_shift}_{shifting_threshold}.pkl')

In [ ]:
validation_imgs = os.listdir(f'{base_path}/dataset1_files/all_dataset1_imgs_validation_0')
wsi_1_validation_imgs = 0
wsi_2_validation_imgs = 0
for img in validation_imgs:
    if img.split('.png')[0] in wsi_1_ids:
        wsi_1_validation_imgs += 1
    else:
        wsi_2_validation_imgs += 1
print(wsi_1_validation_imgs, wsi_2_validation_imgs)

In [ ]:
def load_img(img_dir, img_id):
  img = cv2.imread(f"{img_dir}/{img_id}.png")
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img
def get_annotated_mask(imgs_dir, annots_dir, base_img_id):
  orig_img = load_img(imgs_dir, base_img_id)
  img_height, img_width, _ = orig_img.shape
  with open(f'{annots_dir}/{base_img_id}.pkl', 'rb') as f:
    tgt_annots = pickle.load(f)
  img_mask = np.zeros((img_height, img_width))
  for tgt_annot in tgt_annots:
    coords = [[x, y] for x, y in zip(tgt_annot['segmentation'][0][::2], tgt_annot['segmentation'][0][1::2])]
    cv2.fillPoly(img_mask, pts=[np.array(coords)], color=1)
  return img_mask

In [ ]:
import time
if generate_masks:
  train_modes = ['train', 'validation']
  for i in range(1):
    start_time = time.time()
    for train_mode in train_modes:
      orig_img_dir = f'{base_path}/dataset1_files/all_dataset1_imgs_merged_{train_mode}_{i}'
      orig_annots_dir = f'{base_path}/dataset1_files/all_dataset1_annotations_merged_{train_mode}_{i}'
      output_mask_dir = f'{base_path}/dataset1_files/all_dataset1_masks_merged_{train_mode}_{i}'
      all_orig_imgs = os.listdir(orig_img_dir)
      for orig_img in all_orig_imgs:
        orig_img_id = orig_img.split('.png')[0]
        annotated_mask = get_annotated_mask(orig_img_dir, orig_annots_dir, orig_img_id)
        cv2.imwrite(f'{output_mask_dir}/{orig_img_id}.png', annotated_mask)
    print(f'Finished generating masks for fold {i} in {float(time.time()-start_time)/60} minutes')
    start_time = time.time()

In [ ]:
tgt_img_id = 'b8db704134ac_BR_0.75'
train_mode = 'train'
i = 0
orig_img = get_annotated_img(f'{base_path}/dataset1_files/all_dataset1_imgs_merged_{train_mode}_{i}', f'{base_path}/dataset1_files/all_dataset1_annotations_merged_{train_mode}_{i}', tgt_img_id)
# img_mask = cv2.imread(f'{base_path}/dataset1_files/all_dataset1_masks_merged_{train_mode}_{i}/{tgt_img_id}.png', cv2.IMREAD_GRAYSCALE)
plt.imshow(orig_img)
plt.show()
# plt.imshow(img_mask)
# plt.show()

In [ ]:
with open('/home/ec2-user/hubmap-hacking-the-human-vasculature/dataset1_files/all_dataset1_annotations_merged_train_0/bd090bb1b654.pkl', 'rb') as f:
    data = pickle.load(f)

In [ ]:
annot_files = os.listdir('/home/ec2-user/hubmap-hacking-the-human-vasculature/dataset1_files/all_dataset1_annotations_merged_train_0')
no_mask_annots = []
for annot_file in annot_files:
    with open(f'/home/ec2-user/hubmap-hacking-the-human-vasculature/dataset1_files/all_dataset1_annotations_merged_train_0/{annot_file}', 'rb') as f:
        data = pickle.load(f)
    if len(data)==0:
        no_mask_annots.append(annot_file)

In [ ]:
for no_mask_annot in no_mask_annots:
    img_id = no_mask_annot.split('.pkl')[0]
    if os.path.exists(f'/home/ec2-user/hubmap-hacking-the-human-vasculature/dataset1_files/all_dataset1_annotations_merged_train_0/{img_id}.pkl'):
        os.remove(f'/home/ec2-user/hubmap-hacking-the-human-vasculature/dataset1_files/all_dataset1_annotations_merged_train_0/{img_id}.pkl')
    if os.path.exists(f'/home/ec2-user/hubmap-hacking-the-human-vasculature/dataset1_files/all_dataset1_imgs_merged_train_0/{img_id}.png'):
        os.remove(f'/home/ec2-user/hubmap-hacking-the-human-vasculature/dataset1_files/all_dataset1_imgs_merged_train_0/{img_id}.png')
